## Import libraries

In [1]:
from geo_utils import *

### Load settlement data, select the hamlet part

In [2]:
# Specify country code so all output file will be labeled correctly
country_alpha_3_code = 'ZMB'

# Specify UTM zone EPSG code for this country, so that projection operation in this notebook knows the target CRS
country_utm_epsg = 'epsg:32735'

# africa albers equal area conic
albers_equal_area = 'epsg:102022' # not found

default_pcs = country_utm_epsg

---
Function `explode_geometry`

Parameters: 
- `gdf`, *GeoDataframe* 
- `geom_colum`, *str*   (default = 'geometry') 
- `id_column`, *str*   (default = None, must specify if using drop_duplicates) 
- `drop_duplicates`, *boolean*   (default = False) 

Returns: 
- A GeoDataframe with geometry column exploded into single-polygon/single-linestring objects. Use drop_duplicates in combination with id_column to keep only the the largest shape among the shapes with the same original id.

---

---
Function `add_buffer_column`

Parameters: 
- `gdf`, *GeoDataframe* 
- `buffer_radius`, *int*  (in meters)
- `geom_column`, *str*   (default = 'geometry') 
- `buffer_shape`, *str*   (default = 'round', options: 'round','square','flat') 
- `proj2`, *str* (default = None, options: any valid crs)
- `replace`, *boolean* (default = False)
- `return_new_column`, *boolean* (default = False)

Returns: 
- A GeoDataframe with a new buffer column. Set the geometry from which to buffer using `geom_column`. Set buffer radius with `buffer_radius`, units are in meters, the radius can be negative for shrinking shapes, though multiple shapes may be created as a result. Control shape of buffer with `buffer_shape`. If you want the buffer to be based on a projection other than current projection, use `proj2`, it will not affect the projection of the input dataframe, it only applies to the new buffer column. If you want to replace the main geometry column with the newly created buffer column, set `replace` to True. If you want to get the name of the newly created buffer column, set `return_new_column` to True. 

---

In [3]:
# Read in the shapefile of all settlements
settlement_extents = gpd.read_feather('./data/ZMB_grid3_settlement_extents_20201222.feather').to_crs(default_pcs)

# Filter down to hamlet settlements (the focus of this false positive prediction workflow)
hamlet_settlements = settlement_extents.query('type == "hamlet"')[['mgrs_code','geometry']]

# Convert the default "multipolygon" geometry type to (single)"polygon" geometry type
hamlet_settlements = explode_geometry(hamlet_settlements, id_column = 'mgrs_code', drop_duplicates = True)

hamlet_settlements = add_buffer_column(hamlet_settlements, -40, proj2=None, replace=True)

hamlet_settlements = explode_geometry(hamlet_settlements, id_column = 'mgrs_code', drop_duplicates = True)

### Feature 1:  Google building presence

---
Function `read_csv_as_gpd`

Parameters: 
- `df_or_filepath`, *GeoDataframe or str*
- `id_column`, *str*   (default = None, if not specified, a UUID column will be generated) 
- `lon_lat_columns`, *list of str*   (default = \[\]) 
- `attribute_columns`, *list of str*  (default = \[\]) 
- `drop_rows_where_duplicates_in_columns`, *list of str*  (default = \[\]) 
- `keep_which_if_duplicates`, *str*  (default = 'first', options: 'first','last')
- `drop_rows_where_nan_in_columns`, *list of str*  (default = \[\]) 

Returns: 
- A GeoDataframe loaded from a CSV file. Use `drop_rows_where_duplicates_in_columns` in combination with `id_column` and `keep_which_if_duplicates` to keep control the behavior when duplicates are detected in certain columns. Use `drop_rows_where_nan_in_columns` to control the dropping of rows with missing values.

---

---
Function `left_spatial_join`

Parameters: 
- `gdf1`, *GeoDataframe*
- `gdf2`, *GeoDataframe*   

Returns: 
- A GeoDataframe resulting from the left spatial join of two input GeoDataframes. Spatial join operation uses "intersection", right index dropped.

---

In [4]:
google_buildings_layer = read_csv_as_gpd('./data/ZMB_google_buildings_footprint.csv', 
                                          id_column = None, 
                                          lon_lat_columns = ['longitude','latitude'], 
                                          attribute_columns = ['area_in_meters','confidence'],
                                          drop_rows_where_duplicates_in_columns = ['longitude','latitude']
                                        ).to_crs(default_pcs)


In [5]:
google_buildings_layer = left_spatial_join(google_buildings_layer, hamlet_settlements[['mgrs_code','geometry']])

settlements_with_google_builing = google_buildings_layer['mgrs_code'].dropna().unique().tolist()

hamlet_settlements.loc[hamlet_settlements['mgrs_code'].isin(settlements_with_google_builing),'google_value'] = 1
hamlet_settlements['google_value'] = hamlet_settlements['google_value'].fillna(0)

### Feature 2: Survey building presence

In [6]:
building_presence_layer = read_csv_as_gpd('./data/ZM_geos_L1_2019.csv', 
                                          id_column = 'sid', 
                                          lon_lat_columns = ['lon','lat'], 
                                          attribute_columns = ['bp'], 
                                          drop_rows_where_duplicates_in_columns = ['observer','lat','lon'], 
                                          drop_rows_where_nan_in_columns = ['lat','lon','bp']
                                         ).to_crs(default_pcs)

In [7]:
building_presence_layer = add_buffer_column(building_presence_layer, buffer_radius = 100, buffer_shape = 'square', replace=True)

hamlet_settlements = left_spatial_join(hamlet_settlements, building_presence_layer[['bp','geometry']])

hamlet_settlements['bp_value'] = hamlet_settlements['bp'].map({'Y':1,'N':0})

hamlet_settlements = hamlet_settlements.drop('bp', axis=1)

### Feature 3: HRSL

---
Function `add_centroid_column`

Parameters: 
- `gdf`, *GeoDataframe* 
- `geom_column`, *str*   (default = 'geometry') 
- `proj2`, *str* (default = None, options: any valid crs)
- `replace`, *boolean* (default = False)
- `return_new_column`, *boolean* (default = False)

Returns: 
- A GeoDataframe with a new centroid column. Set the geometry from which to calculate centroid using `geom_column`. If you want the centroid to be based on a projection other than current projection, use `proj2`, it will not affect the projection of the input dataframe, it only applies to the new centroid column. If you want to replace the main geometry column with the newly created centroid column, set `replace` to True. If you want to get the name of the newly created centroid column, set `return_new_column` to True. 

---

In [8]:
hamlet_settlements, centroid_column = add_centroid_column(hamlet_settlements, return_new_column=True)

# Since HRSL data is in GCS, convert just the centroid column to WGS84 to facilitate pixel lookup in HRSL raster
hamlet_settlements[centroid_column] = hamlet_settlements[centroid_column].to_crs('epsg:4326')

hamlet_settlements = get_raster_point_value(hamlet_settlements, centroid_column, raster_filepath = './data/ZMB_HRSL_hasim_20211222.tif', new_column = 'hrsl_value')

### Feature 4: WSF

---
Function `add_covering_geotiff_column`

Parameters: 
- `gdf`, *GeoDataframe* 
- `geom_column`, *str* 
- `geotiff_filepath_column`, *str*
- `geotiff_filepath_list`, *list of str* 

Returns: 
- A GeoDataframe with a new column storing the filepaths of the geotiff that cover the shapes in each row. Set the geometry with which to find geotiff using `geom_column`. Set the name of the new geotiff_filepath column with `geotiff_filepath_column`. Provide the filepaths of the candidate geotiffs in `geotiff_filepath_list`. All parameters need to be explicitly specified.

---

---
Function `get_raster_value_distribution`

Parameters: 
- `gdf`, *GeoDataframe* 
- `id_column`, *str*
- `geom_column`, *str* 
- `geotiff_filepath_column`, *str*
- `geotiff_filepath_list`, *list of str* 
- `code_to_label_mapping`, *list of str* 
- `label_marker`, *list of str* 
- `normalize`, *boolean* 

Returns: 
- A GeoDataframe with new columns corresponding to the distribution of different codes in the covering raster image. Specify which raster geotiff is covering the geometry with `geotiff_filepath_column`. Provide the filepaths of the candidate geotiffs in `geotiff_filepath_list`. Use `code_to_label_mapping` to specify the mapping from numerical codes to human-readable labels, this may vary from one standard to another. `label_marker` is a prefix to all the newly-created column, so as to mark which raster these columns are derived from. `normalize` controls whether the values in the columns are proportion or absolute count of pixels. All parameters need to be explicitly specified.

---

In [9]:
# Since WSF is in GCS, converting the geometry of the settlements to WGS84 to facilitate spatial matching
hamlet_settlements.to_crs('epsg:4326', inplace=True)

In [10]:
wsf_2019_geotiff_filepath_list = glob('./data/WSF2019/*.tif')

In [11]:
hamlet_settlements = add_covering_geotiff_column(hamlet_settlements, 
                                                 geom_column = 'geometry',
                                                 geotiff_filepath_column = 'wsf2019_geotiff_filepath',
                                                 geotiff_filepath_list = wsf_2019_geotiff_filepath_list)

In [12]:
hamlet_settlements = get_raster_value_distribution(hamlet_settlements, 
                                                   id_column = 'mgrs_code',
                                                   geom_column = 'geometry',
                                                   geotiff_filepath_column = 'wsf2019_geotiff_filepath',
                                                   geotiff_filepath_list = wsf_2019_geotiff_filepath_list, 
                                                   code_to_label_mapping = {0:'no_settlement',255:'settlement'},
                                                   label_marker = 'wsf2019',
                                                   normalize = True)

100%|███████████████████████████████████████████| 56/56 [01:23<00:00,  1.49s/it]


---
Function `drop_bounds`

Parameters: 
- `gdf`, *GeoDataframe* 
- `geom_column`, *str* 

Returns: 
- A GeoDataframe with geometry bounds columns dropped, these include the columns that start with a geometry column name and end with `minx`, `maxx`, `miny`, or `maxy`. This is a utility function to help similify output of `get_raster_value_distribution` function.

---

In [13]:
hamlet_settlements = drop_bounds_columns(hamlet_settlements, geom_column='geometry')

In [14]:
hamlet_settlements['wsf_value'] = (hamlet_settlements['wsf2019__settlement']>0).astype(int)

In [15]:
hamlet_settlements.to_crs(default_pcs, inplace=True)

### Create true positive  / false positive samples based on agreement of sources

In [16]:
all_agree_positive = hamlet_settlements.query('bp_value == 1 & google_value == 1 & hrsl_value == 1 & wsf_value == 1')
print('There are',len(all_agree_positive),'true positive samples.')

There are 618 true positive samples.


In [17]:
all_agree_negative = hamlet_settlements.query('bp_value == 0 & google_value == 0 & hrsl_value == 0 & wsf_value == 0')
print('There are',len(all_agree_negative),'false positive samples.')

There are 148 false positive samples.


### Combine manually labeled training data and ruled-based new training data

In [18]:
training_data = gpd.read_file('./data/ZMB_grid3_training_points.geojson')
training_data = training_data.query('type == "hamlet"')[['mgrs_code','false_posi']]
training_data['false_posi'] = training_data['false_posi'].apply(int)

In [19]:
all_agree_positive = all_agree_positive[['mgrs_code']]
all_agree_positive['false_posi'] = 0
all_agree_negative = all_agree_negative[['mgrs_code']]
all_agree_negative['false_posi'] = 1
rule_based_new_training_data = all_agree_positive.append(all_agree_negative, ignore_index = True)

In [20]:
expanded_training_data = training_data.append(rule_based_new_training_data, ignore_index = True)

In [21]:
hamlet_settlements = hamlet_settlements[['mgrs_code','geometry','wsf2019_geotiff_filepath','wsf2019__no_settlement','wsf2019__settlement']]
expanded_training_data = hamlet_settlements.merge(expanded_training_data, on='mgrs_code', how='inner')

In [ ]:
expanded_training_data.to_feather(X './data/expanded_training_data_v20220509.feather')

## Additional features for classification

In [23]:
gdf = gpd.read_feather('./data/expanded_training_data_v20220509.feather')

In [24]:
gdf = add_buffer_column(gdf, 50)
gdf = add_buffer_column(gdf, 500)
gdf = add_buffer_column(gdf, 5000)

---
Function `add_intersection_count_column`

Parameters:
- `gdf`, *GeoDataframe* 
- `uuid_column`, *str* 
- `buffer_column`, *str* 
- `feature_layer`, *str* 
- `new_column`, *str* 
- `feature_geom_column`, *str* (default = 'geometry')

Returns: 
- A GeoDataframe with a new column that counts the feature geometries within the buffer of main geometry. `gdf` is the main GeoDataframe that has a buffer column, specified by `buffer_column`. `feature_layer` is the other GEoDataframe with features, by default the 'geometry' column of the feature layer will be used but it can be changed. `new_column` controls the name of the newly-created column.

---

### Feature 1: Road count

In [25]:
fb_road_network = gpd.read_feather('./data/ZMB_mapwithai_road_data_v20200729.feather').to_crs(default_pcs)

osm_road_network = gpd.read_feather('./data/ZMB_osm_all_road_network_v20220212.feather').to_crs(default_pcs)

for col in gdf.columns:
    if col.startswith('geometry_buffer_'):
        gdf = add_intersection_count_column(gdf, 'mgrs_code', col, fb_road_network, 'fb_road_within_'+col.replace('geometry_buffer_',''))

for col in gdf.columns:
    if col.startswith('geometry_buffer_'):
        gdf = add_intersection_count_column(gdf, 'mgrs_code', col, osm_road_network, 'osm_road_within_'+col.replace('geometry_buffer_',''))

### Feature 2: OSM landuse count

In [26]:
osm_landuse = gpd.read_feather('./data/ZMB_osm_landuse_v20220418.feather').to_crs(default_pcs)

common_osm_landuse_categories = ['farmland', 'residential', 'forest']

for col in gdf.columns:
    if col.startswith('geometry_buffer_'):
        for landuse_category in common_osm_landuse_categories:
            gdf = add_intersection_count_column(gdf, 'mgrs_code', col, osm_landuse[osm_landuse['landuse']==landuse_category], landuse_category+'_land_use_within_'+col.replace('geometry_buffer_',''))

### Feature 3: Distance to nearest settlement

---
Function `add_distance_to_nearest_neighbor_column`

Parameters:
- `gdf`, *GeoDataframe* 
- `geom_centroid_column`, *str* 
- `new_column`, *str* 
- `rounding`, *str* (default = 0)

Returns: 
- A GeoDataframe with a new column that calculate the distance from this geometry to the nearest geometry within the same GeoDataframe. Use `geom_centroid_column` to specify which geometry to do nearest distance calculation. `new_column` controls the name of the newly-created column. The distance is measured in meters and rounded by default, but can be changed with `rounding` parameter. 

---

In [27]:
settlement_extents = gpd.read_feather('./data/ZMB_grid3_settlement_extents_20201222.feather').to_crs(default_pcs)
settlement_extents, centroid_column = add_centroid_column(settlement_extents, return_new_column=True)
settlement_extents[centroid_column] = settlement_extents[centroid_column].to_crs('epsg:4326')

settlement_extents = add_distance_to_nearest_neighbor_column(settlement_extents, centroid_column, 'distance_to_nearest_settlement')

gdf = gdf.merge(settlement_extents[['mgrs_code','distance_to_nearest_settlement']], on='mgrs_code', how='left')

### Feature 4: ESA landcover

In [28]:
gdf.to_crs('epsg:4326', inplace=True)

In [29]:
esalc_geotiff_filepath_list = glob('./data/ESA_LC/*_Map.tif')

In [30]:
gdf = add_covering_geotiff_column(gdf, 
                                  geom_column = 'geometry',
                                  geotiff_filepath_column = 'esalc_geotiff_filepath',
                                  geotiff_filepath_list = esalc_geotiff_filepath_list)

gdf = gdf.dropna(subset=['esalc_geotiff_filepath'])

gdf = get_raster_value_distribution(gdf, 
                                    geom_column = 'geometry',
                                    uuid_column = 'mgrs_code',
                                    geotiff_filepath_column = 'esalc_geotiff_filepath',
                                    geotiff_filepath_list = esalc_geotiff_filepath_list, 
                                    code_to_label_mapping = {10: 'forest',
                                                             20: 'shrubland',
                                                             30: 'grassland',
                                                             40: 'cropland',
                                                             50: 'built',
                                                             60: 'bare',
                                                             70: 'ice',
                                                             80: 'water',
                                                             90: 'wetland',
                                                             100: 'tundra',}  ,
                                    label_marker = 'esalc',
                                    normalize = True)

gdf = drop_bounds_columns(gdf)

77 rows does not have a matched geotiff.


100%|███████████████████████████████████████████| 12/12 [00:12<00:00,  1.07s/it]


In [31]:
# Drop land cover ice to avoid perfect colinearity in analysis, also because no where in target country has ice cover
gdf = gdf.drop('esalc__ice', axis=1)

### Feature 5: Google building footprint confidence

---
Function `get_groupby_stats_df`

Parameters:
- `data`, *Dataframe or GeoDataframe* 
- `groupby_column`, *str* 
- `stats_map`, *dict* 

Returns: 
- A Dataframe with statistics of the provided features, a simple wrapper around Pandas `groupby` function.

---

In [32]:
google_building_groupby_stats_df = get_groupby_stats_df(google_buildings_layer, 'mgrs_code', {'area_in_meters':[min,max,np.mean,np.median],'confidence':[min,max,np.mean,np.median]})

gdf = pd.merge(gdf, google_building_groupby_stats_df, left_on = 'mgrs_code', right_index = True, how='left')

---
Function `get_most_correlated_feature`

Parameters:
- `data`, *Dataframe or GeoDataframe* 
- `target`, *str* 
- `features`, *list of str* 

Returns: 
- The name of feature that is most correlated with the target, as measured by Pearson R. This is a simple utility function for choosing one feature when several features are highly correlated with each other.

---

In [33]:
get_most_correlated_feature(gdf, target = 'false_posi', features = ['area_in_meters__min', 'area_in_meters__max', 'area_in_meters__mean', 'area_in_meters__median'])

'area_in_meters__min'

In [34]:
get_most_correlated_feature(gdf, target = 'false_posi', features = ['confidence__min', 'confidence__max', 'confidence__mean', 'confidence__median'])

'confidence__max'

In [35]:
gdf = gdf.drop(['area_in_meters__max', 'area_in_meters__mean', 'area_in_meters__median']+['confidence__min', 'confidence__mean', 'confidence__median'], axis=1)

In [36]:
gdf = gdf.fillna(0)

### Eliminate non-features and non-indicative features

In [37]:
gdf = gdf[['mgrs_code', 'false_posi',
            'fb_road_within_50', 'fb_road_within_500', 'fb_road_within_5k',
            'osm_road_within_50', 'osm_road_within_500', 'osm_road_within_5k',

            'farmland_land_use_within_50', 'residential_land_use_within_50', 'forest_land_use_within_50',
            'farmland_land_use_within_500', 'residential_land_use_within_500', 'forest_land_use_within_500',
            'farmland_land_use_within_5k', 'residential_land_use_within_5k', 'forest_land_use_within_5k',

            'esalc__forest', 'esalc__shrubland', 'esalc__grassland',
            'esalc__cropland', 'esalc__built', 'esalc__bare', 'esalc__water',
            'esalc__wetland', 'esalc__tundra', 

            'distance_to_nearest_settlement', 'area_in_meters__min', 'confidence__max']]

gdf = gdf.set_index('mgrs_code')

gdf = gdf.drop(  gdf.nunique()[gdf.nunique() == 1].index.tolist(), axis=1)

### For buffer features, convert from within values to range values

---
Function `within_value_to_range_value`

Parameters:
- `gdf`, *GeoDataframe* 
- `buffer_radius_markers`, *str* 

Returns: 
- A GeoDataframe with new columns tracking the count of features in the ring areas around main geometries. For example, the number of features in the ring area that is at least 500 meters away but at most 5000 meters away from a settlement. This is based on the observation that number of features within 5000 meters must include the number of features within 500 meters, which creates collinearity that hurts prediction models. Thus, this function calculates the count in a specific range instead of the count within a radius.

---

In [38]:
gdf = within_value_to_range_value(gdf, buffer_radius_markers = ['within_50','within_500','within_5k'])

### Reorder feature columns

In [39]:
gdf = gdf[['false_posi']+sorted([col for col in gdf.columns if col != 'false_posi'])]

### Export dataframe

In [ ]:
gdf.to_csv('./data/cleaned_training_data_v20220509.csv')